# Network Correlation and Covariance Analysis

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import scipy.spatial.distance as spd
import networkx as nx

def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(1/2)

def distanceWBoundary(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    dists = []
    dists.append(distance(x1,y1,x2,y2))
    dists.append(distance(x1,y1,x2-1,y2))
    dists.append(distance(x1,y1,x2+1,y2))
    dists.append(distance(x1,y1,x2,y2-1))
    dists.append(distance(x1,y1,x2,y2+1))
    dists.append(distance(x1,y1,x2-1,y2-1))
    dists.append(distance(x1,y1,x2-1,y2+1))
    dists.append(distance(x1,y1,x2+1,y2-1))
    dists.append(distance(x1,y1,x2+1,y2+1))
    return min(dists)

def getResults(topoNum):
    eucSumArr = []
    netSumArr = []
    eucXsArr = []
    eucYsArr = []
    netXsArr = []
    netYsArr = []
    
    if(netNum == 6 and topoNum == 4):
        return
    print(topoNum)
    num = "%02d"%(topoNum+1)
    
    #Load Point Patterns and save into pointDict
    f = open("Networks/"+netDict[netNum]+'-'+num+'.txt','r')
    lines = f.readlines()
    f.close()
    if(netNum == 6):
        if(topoNum == 0):
            coords = np.array([x.rstrip().split('\t') for x in lines if x.rstrip().split(' ') != ['']]).astype(float)
        else:
            coords = [(float(x.lstrip().rstrip().split(' ')[0]),float(x.lstrip().rstrip().split(' ')[-1])) for x in lines if x.rstrip().split(' ') != ['']]
    else:
        coords = np.array([x.rstrip().split('\t') for x in lines if x.rstrip().split(' ') != ['']]).astype(float)
    
    if(netNum > 2):
        coords = coords/100
        
    #Calculate Euclidean distances between all pairs of points
    eucDistArr = spd.squareform(spd.pdist(coords,distanceWBoundary))
    print("Found Euclidean Distances")

    G = nx.read_edgelist("Networks/"+netDict[netNum]+'_'+num+'_'+netType.lower()+'_edgelist.txt', nodetype = int, data = [('distance',float)])
    if(netNum > 2):
        for edge in G.edges():
            G.edges()[edge]['distance']/=100
    G = G.subgraph(max(list(nx.connected_components(G)),key = len))
    nodes = list(G.nodes())
    distDict = dict(nx.all_pairs_dijkstra_path_length(G,weight = 'distance'))
    distArr = np.zeros((len(nodes),len(nodes)))
    #eucDistArr = np.zeros((len(G),len(G)))
    for i in range(len(nodes)):
        for j in range(i):
            if(i in distDict[j]):
                distArr[i][j] = distDict[i][j]
                distArr[j][i] = distDict[j][i]
            else:
                distArr[i][j] = np.inf
                distArr[j][i] = np.inf
    del distDict

    #Save Densities/Co-densities
    dist = 0.1
    for measDist in measDists:
        print(measDist)
        eucSumArr.append(list(np.sum(eucDistArr<=measDist,axis=0)))
        netSumArr.append(list(np.sum(distArr<=measDist,axis=0)))
        
    for m in range(len(measDists)-1):    
        print(np.round(measDists[m],decimals=3))
        nodePairs = (eucDistArr >= measDists[m]) & (eucDistArr <= measDists[m+1])
        nodePairsNet = (distArr >= measDists[m]) & (distArr <= measDists[m+1])
        eucDists = np.sum(eucDistArr<=dist,axis=0)
        dists = np.sum(distArr<=dist,axis=0)
        xs = []
        ys = []
        xsNet = []
        ysNet = []
        for i in range(len(nodes)):
            densities = list(eucDists[nodePairs[i]])
            xs += [eucDists[i]]*len(densities)
            ys += densities
            densitiesNet = list(dists[nodePairsNet[i]]) 
            xsNet += [dists[i]]*len(densitiesNet)
            ysNet += densitiesNet
                
        eucXsArr.append(xs)
        eucYsArr.append(ys)
        netXsArr.append(xsNet)
        netYsArr.append(ysNet)
            
    return eucSumArr,netSumArr,eucXsArr,eucYsArr,netXsArr,netYsArr


netDict = {0:'HIP', 1:'Poisson', 2:'RSA', 3:'Stealthy0.20', 4:'Stealthy0.40',5:'Stealthy0.49',6:"Gaussian"}
netNameDict = {0:'HIP', 1:'Poisson', 2:'RSA', 3:'Stealthy0.20', 4:'Stealthy0.40',5:'Stealthy0.49',6:"Non-stealthy Hyperuniform"}
netType = "delaunay"
#Real Inputs
#measDists = np.arange(0.01,0.5,0.001)

#Test Inputs
measDists = [0.1,0.11]
for netNum in [0]:#range(1):
    print(netDict[netNum])
    if(netNum == 1):
        #numTopos = 6
        numTopos = 1
    else:
        #numTopos = 10
        numTopos = 1
       
    eucSumRes = [[] for _ in range(len(measDists))]
    netSumRes = [[] for _ in range(len(measDists))]
    eucXsRes = [[] for _ in range(len(measDists))]
    eucYsRes = [[] for _ in range(len(measDists))]
    netXsRes = [[] for _ in range(len(measDists))]
    netYsRes = [[] for _ in range(len(measDists))]
    
    for topoNum in range(numTopos):
        res = getResults(topoNum)
        for m in range(len(measDists)):
            eucSumRes[m] += res[0][m]
            netSumRes[ri][m] += res[1][m]
            if(m != len(measDists)-1):
                eucXsRes[m] += res[2][m]
                eucYsRes[m] += res[3][m]
                netXsRes[ri][m] += res[4][m]
                netYsRes[ri][m] += res[5][m]
                
    corrs = []
    for m in range(len(measDists)):
        corrs.append(sp.pearsonr(eucSumRes[m],netSumRes[m])[0])
    correlationResDF = pd.DataFrame(corrs,columns=measDists)
    
    covarianceResDF = pd.DataFrame(columns=measDists)
    covs = []
    netCovs = []
    for m in range(len(measDists)):
        if(len(eucXsRes[m])>2):
            covs.append(sp.pearsonr(eucXsRes[m],eucYsRes[m])[0])
        else:
            covs.append(np.nan)
        if(len(netXsRes[R][m])>2):
            netCovs.append(sp.pearsonr(netXsRes[R][m],netYsRes[R][m])[0])
        else:
            netCovs.append(np.nan)
    covarianceResDF.loc["Euclidean"] = covs
    covarianceResDF.loc["Network"] = netCovs

HIP
0
Found Euclidean Distances


FileNotFoundError: [Errno 2] No such file or directory: 'Networks/HIP/HIP/HIP_01_delaunay_edgelist.txt'